This was initially cloned by https://github.com/rahulbot/notebook-examples and modified for the purpose of 
doing an assignment on MAS.500 class.

Fetching Webpages with the Requests Library To Demonstrate
===========================================
SqlAlchemy helps you use a database to store and retrieve information from python.  It abstracts the specific storage engine from te way you use it - so it doesn't care if you end up using MySQL, SQLite, or whatever else. In addition, you can use core and the object-relational mapper (ORM) to avoid writing any SQL at all.  The [SQLAlchemy homepage](http://www.sqlalchemy.org/) has lots of good examples and full documentation.

In [29]:
import mediacloud
from ConfigParser import ConfigParser

config = ConfigParser()
config.read('config.cfg') # git ignore
api_key = config.get('Media Cloud','my_api')

def connect_to_media_cloud(_api_token):
    mc = mediacloud.api.MediaCloud(_api_token)
    
    if not mc.verifyAuthToken():
        print 'media cloud authentication failed'
        return None
    else:
        print 'connected to media cloud'
        return mc

mc = connect_to_media_cloud(api_key)

def sentence_count(_keyword):
    '''
    simplified version of sentenceCount function. Gets the number
    from the start of this year(2015) until today.
    '''
    result = mc.sentenceCount(_keyword,
                         solr_filter =[
                        mc.publish_date_query(
                        datetime.date(2015,1,1),
                        datetime.date.today()),
                        'media_sets_id:1'])
    return result['count']


# test sentence count
japan_count = sentence_count('japan') 

print japan_count

connected to media cloud
59223


In [3]:
from sqlalchemy import *
import datetime

## Basic SQL Generation
The core library generates SQL for you.  Read more about it on their [expression language tutorial page](http://docs.sqlalchemy.org/en/rel_1_0/core/index.html). Below are some basic examples.

### Creating a Table
Read more about [defining and creating tables](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#define-and-create-tables).

In [39]:
# add `echo=True` to see log statements of all the SQL that is generated
engine = create_engine('sqlite:///:memory:') # just save the db in memory for now (ie. not on disk)
metadata = MetaData()
# define a table to use
queries = Table('queries', metadata,
    Column('id', Integer, primary_key=True),
    Column('keywords', String(400), nullable=False),
    Column('count',Integer),
    Column('timestamp', DateTime, default=datetime.datetime.now),
)
metadata.create_all(engine) # and create the tables in the database

### Inserting Data
Read more about generating [SQL insert statements](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#insert-expressions).

In [40]:
insert_stmt = queries.insert()
str(insert_stmt) # see an example of what this will do

'INSERT INTO queries (id, keywords, count, timestamp) VALUES (:id, :keywords, :count, :timestamp)'

In [41]:
insert_stmt = queries.insert().values(keywords="puppies",count=sentence_count("puppies"))
str(insert_stmt)

'INSERT INTO queries (keywords, count, timestamp) VALUES (:keywords, :count, :timestamp)'

In [42]:
db_conn = engine.connect()
result = db_conn.execute(insert_stmt)
result.inserted_primary_key # print out the primary key it was assigned

[1]

In [43]:
insert_stmt = queries.insert().values(keywords="kittens",count=sentence_count("kittens"))
result = db_conn.execute(insert_stmt)
result.inserted_primary_key # print out the primary key it was assigned

[2]

### Retrieving Data
Read more about using [SQL select statments](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#selecting).

In [44]:
from sqlalchemy.sql import select
select_stmt = select([queries])
results = db_conn.execute(select_stmt)
for row in results:
    print row

(1, u'puppies', 5678, datetime.datetime(2015, 12, 2, 20, 6, 57, 460742))
(2, u'kittens', 2804, datetime.datetime(2015, 12, 2, 20, 7, 3, 842666))


In [10]:
select_stmt = select([queries]).where(queries.c.id==1)
for row in db_conn.execute(select_stmt):
    print row

(1, u'puppies', datetime.datetime(2015, 12, 2, 19, 52, 12, 10677))


In [45]:
select_stmt = select([queries]).where(queries.c.keywords.like('p%'))
for row in db_conn.execute(select_stmt):
    print row

(1, u'puppies', 5678, datetime.datetime(2015, 12, 2, 20, 6, 57, 460742))


Homework 1 : Add a select and for loop for getting the sum of the sentence count

In [47]:
#sum of the 'counts'
total_sentence_count = 0
select_stmt = select([queries])
results = db_conn.execute(select_stmt)
for row in results:
    total_sentence_count += row[2]

print total_sentence_count

8482


## ORM
You can use their ORM library to handle the translation into full-fledged python objects.  This can help you build the Model for you [MVC](https://en.wikipedia.org/wiki/Model–view–controller) solution.

In [12]:
import datetime
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
Base = declarative_base()

### Creating a class mapping
Read more about [creating a mapping](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#declare-a-mapping).

In [14]:
class Query(Base):
    __tablename__ = 'queries'
    id = Column(Integer, primary_key=True)
    keywords = Column(String(400))
    timestamp = Column(DateTime,default=datetime.datetime.now)
    count = Column(Integer)
    def __repr__(self):
        return "<Query(keywords='%s')>" % (self.keywords)
Query.__table__

Table('queries', MetaData(bind=None), Column('id', Integer(), table=<queries>, primary_key=True, nullable=False), Column('keywords', String(length=400), table=<queries>), Column('timestamp', DateTime(), table=<queries>, default=ColumnDefault(<function <lambda> at 0x7f67cf3caaa0>)), Column('count', Integer(), table=<queries>), schema=None)

### Creating a connection and session
Read more about [creating this stuff](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#creating-a-session).

In [15]:
engine = create_engine('sqlite:///:memory:') # just save the db in memory for now (ie. not on disk)
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
# a stack of queries
my_session = Session()

### Inserting Data
Read more about [inserting data with an ORM](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#adding-new-objects).

In [48]:
# add a instance containing
query = Query(keywords="iguana")

query.count =  sentence_count(query.keywords)
query

<Query(keywords='iguana')>

In [17]:
my_session.add(query)
my_session.commit()
query.id

1

### Retrieving Data
Read more about [retrieving data from the db](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#querying) via an ORM class.

In [18]:
for q in my_session.query(Query).order_by(Query.timestamp):
    print q.count

357


In [21]:
query1 = Query(keywords="robot")
query2 = Query(keywords="puppy")

for q in [query1,query2]:
    q.count = sentence_count(q.keywords)

my_session.add_all([query1,query2])
my_session.commit()

In [50]:
# get the sum
count_sum = 0
for q in my_session.query(Query).order_by(Query.count):
    print q
    count_sum += q.count

print count_sum

<Query(keywords='iguana')>
<Query(keywords='puppy')>
<Query(keywords='robot')>
25394


In [30]:
for q in my_session.query(Query).filter(Query.keywords.like('p%')):
    print q

<Query(keywords='puppy')>
